## Setup experiment

### Import dependencies

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

## %pip install transformers trl wandb


In [3]:

## Python >= 3.8  


In [4]:

# !pip install transformers
## !pip install wandb
# !pip install pandas
#!pip install datasets
#!pip install accelerate
# !pip install tyro
#!pip install trl

In [5]:
import torch
from tqdm import tqdm
import pandas as pd
import wandb
import os

tqdm.pandas()

from transformers import pipeline, AutoTokenizer
from datasets import load_dataset
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler

C:\Users\danda\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
C:\Users\danda\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
comet_ml is installed but `COMET_API_KEY` is not set.


### Configuration

In [6]:

config = PPOConfig(
    model_name    = "lvwerra/gpt2-imdb", 
    learning_rate = 1.41e-5,
    ## log_with      = "wandb",
)

sent_kwargs = {
         "return_all_scores": True, 
         "function_to_apply": "none", 
         "batch_size": 16
}


In [7]:
#enable wandb
wandb.init()




wandb: Currently logged in as: turne292 (danielle_turner). Use `wandb login --relogin` to force relogin


## Load data and models

### Load IMDB dataset
The IMDB dataset contains 50k movie review annotated with "positive"/"negative" feedback indicating the sentiment.  We load the IMDB dataset into a DataFrame and filter for comments that are at least 200 characters. Then we tokenize each text and cut it to random size with the `LengthSampler`.

In [8]:
def build_dataset(config, dataset_name="turne292/tldr_sentiment_data_small_v2", input_min_text_length=2, input_max_text_length=8):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.

    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """
    tokenizer           = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token
    
    # load imdb with datasets
    
    ds = load_dataset(dataset_name, split="train")
    ds = ds.filter(lambda x: len(x["text"]), batched=False)
    print(len(ds))

    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode( sample["text"]    )[: input_size()]
        sample["query"]     = tokenizer.decode( sample["input_ids"] )
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds
    

In [9]:

dataset = build_dataset(config)
print(len(dataset))

def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])
    

Found cached dataset csv (C:/Users/danda/.cache/huggingface/datasets/turne292___csv/turne292--tldr_sentiment_data_small_v2-4916c9194fdcb323/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


Filter:   0%|          | 0/47217 [00:00<?, ? examples/s]

47217


Map:   0%|          | 0/47217 [00:00<?, ? examples/s]

47217


In [10]:
dataset[55]

{'text': ' r/relationships\nTITLE: How do I (F22) tell an oblivious roommate (F22) that she is gross?\nPOST: Not sure if this is the right place to post. My roommate just moved in a few weeks ago and I am having trouble...\n\nI already told her directly "Let\'s please make sure all the food is rinsed off the dishes and utensils before we put them in the rack." and "I cleaned the counters today, so let\'s clean up after ourselves every time we cook so we can maintain it."\n\nBut every day there is meat juice, pieces of chicke',
 'label': 'negative',
 'input_ids': tensor([  374,    14, 39468,  5748]),
 'query': ' r/relationships'}

### Load pre-trained GPT2 language models

We load the GPT2 model with a value head and the tokenizer. We load the model twice; the first model is optimized while the second model serves as a reference to calculate the KL-divergence from the starting point. This serves as an additional reward signal in the PPO training to make sure the optimized model does not deviate too much from the original language model.

In [11]:
model     = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)

tokenizer = AutoTokenizer.from_pretrained(config.model_name)

tokenizer.pad_token = tokenizer.eos_token


C:\Users\danda\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


### Initialize PPOTrainer
The `PPOTrainer` takes care of device placement and optimization later on:

In [12]:

ppo_trainer = PPOTrainer(
                   config, 
                   model, 
                   ref_model, 
                   tokenizer, 
                   dataset       = dataset, 
                   data_collator = collator
)


### Load BERT classifier
We load a BERT classifier fine-tuned on the IMDB dataset.

In [13]:

device = ppo_trainer.accelerator.device

if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug
    
sentiment_pipe = pipeline("sentiment-analysis", model="lvwerra/distilbert-imdb", device=device)

The model outputs are the logits for the negative and positive class. We will use the logits for positive class as a reward signal for the language model.

In [14]:

text = "this movie was really bad!!"

sentiment_pipe(text, **sent_kwargs)


C:\Users\danda\anaconda3\Lib\site-packages\transformers\pipelines\text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'NEGATIVE', 'score': 2.3350486755371094},
  {'label': 'POSITIVE', 'score': -2.726576328277588}]]

In [15]:

text = "this movie was really good!!"
sentiment_pipe(text, **sent_kwargs)


[[{'label': 'NEGATIVE', 'score': -2.294790029525757},
  {'label': 'POSITIVE', 'score': 2.557039737701416}]]

### Generation settings
For the response generation we just use sampling and make sure top-k and nucleus sampling are turned off as well as a minimal length.

In [16]:

gen_kwargs = {
         "min_length":   -1, 
         "top_k":       0.0, 
         "top_p":       1.0, 
         "do_sample":  True, 
         "pad_token_id": tokenizer.eos_token_id
}


## Optimize model

### Training loop

The training loop consists of the following main steps:
1. Get the query responses from the policy network (GPT-2)
2. Get sentiments for query/responses from BERT
3. Optimize policy with PPO using the (query, response, reward) triplet

**Training time**

This step takes **~2h** on a V100 GPU with the above specified settings.

In [17]:

output_min_length     = 4
output_max_length     = 16
output_length_sampler = LengthSampler(output_min_length, output_max_length)

wandb.watch(model, log_freq=100)
wandb.watch(ref_model, log_freq=100)

[]

In [18]:

generation_kwargs = {
    "min_length":     -1,
    "top_k":         0.0,
    "top_p":         1.0,
    "do_sample":    True,
    "pad_token_id": tokenizer.eos_token_id,
}


In [19]:

## ppo_trainer.config.steps = 100    ## 20,000
ppo_trainer.config.steps


20000

In [20]:
'''
for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    print(batch)
    input()
'''

'\nfor epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):\n    print(batch)\n    input()\n'

In [24]:


for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]
    print(epoch)

    #### Get response from gpt2
    response_tensors = []
    for query in query_tensors:
        gen_len                             = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response                            = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append( response.squeeze()[-gen_len:] )
    batch["response"] = [ tokenizer.decode(r.squeeze()) for r in response_tensors ]

    #### Compute sentiment score
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [ torch.tensor(output[1]["score"]) for output in pipe_outputs]

    #### Run PPO step
    stats = ppo_trainer.step(
                     query_tensors, 
                     response_tensors, 
                     rewards
    )
    ppo_trainer.log_stats(stats, batch, rewards)
    

0it [00:00, ?it/s]

0


C:\Users\danda\anaconda3\Lib\site-packages\transformers\pipelines\text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
C:\Users\danda\anaconda3\Lib\site-packages\transformers\pipelines\base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
1it [00:20, 20.00s/it]

1


2it [00:38, 19.17s/it]

...

366it [1:52:36, 19.51s/it]

366


367it [1:52:56, 19.50s/it]

367


368it [1:53:15, 18.47s/it]


## Model inspection
Let's inspect some examples from the IMDB dataset. We can use `model_ref` to compare the tuned model `model` against the model before optimisation.

In [22]:
#### get a batch from the dataset
bs                 = 16
game_data          = dict()
dataset.set_format("pandas")
df_batch           = dataset[:].sample(bs)
game_data["query"] = df_batch["query"].tolist()
query_tensors      = df_batch["input_ids"].tolist()

response_tensors_ref, response_tensors = [], []

#### get response from gpt2 and gpt2_ref
for i in range(bs):
    gen_len = output_length_sampler()
    output  = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(output)
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(output)

#### decode responses
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data["response (after)"]  = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

#### sentiment analysis of query/response pairs before/after
texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results

C:\Users\danda\anaconda3\Lib\site-packages\transformers\pipelines\text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
C:\Users\danda\anaconda3\Lib\site-packages\transformers\pipelines\base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


,query,response (before),response (after),rewards (before),rewards (after)
0,r/legaladvice,"), this film attempts to portray how marijuana...",rails through an extremely inventive and ente...,-2.338746,2.737933
1,r/relationships,between cornerstones.<br /><br />The,": good, multiplayer, good games, fun together",0.724979,2.726515
2,r/tifu\n,HBO's Mr. Kavod,`s superb storyline and excellent music.,-0.600709,2.803211
3,r/,h number together (or so I,"g films.) Bob Dylan, and",-0.072393,0.789450
4,r/relations,of the same genius in an independent,"with the original Square inhabited, fulfilling",0.992978,0.455365
5,r/relationships\n,The movie is about hazing and a guy commits su...,"This is a great production, it is great and it...",-0.995547,2.894707
6,r/relationships\nTITLE,LE (10/2/13) * 18 pages<|endoftext|>,"3 is epic, absorbing story that will allow yo...",-0.016736,2.740645
7,r/tifu\nTITLE,"REFERENCE: Spitfires, Spec Ops","Creature/titles are amazing, amazing and",-0.667457,2.728873
8,r/tifu\nTIT,Y favorite likes fanduel 03/12,stood out jerky and amazingly well presented ...,1.412043,2.479338
9,r/relations,"hip among her boyfriend, dean, etc. also, in this",hips and put the whole movie to good use in it...,0.547193,2.114477


Looking at the reward mean/median of the generated sequences we observe a significant difference.

In [23]:
print("mean:")
display(df_results[["rewards (before)", "rewards (after)"]].mean())
print()
print("median:")
display(df_results[["rewards (before)", "rewards (after)"]].median())

mean:


rewards (before)   -0.144184
rewards (after)     2.385100
dtype: float64


median:


rewards (before)   -0.037348
rewards (after)     2.712311
dtype: float64

## Save model
Finally, we save the model and push it to the Hugging Face for later usage.

In [ ]:

## model.save_pretrained(    "gpt2-imdb-pos-v2", push_to_hub=True)
## tokenizer.save_pretrained("gpt2-imdb-pos-v2", push_to_hub=True)

model.save_pretrained(    "./RLHFmodel/gpt2-tldr-pos", push_to_hub=False)
tokenizer.save_pretrained("./RLHFmodel/gpt2-tldr-pos", push_to_hub=False)


In [ ]:

print(tqdm)
